### Wizualizacja danych - projekt
### Scrapper danych pogodowych ze strony `timeanddate.com`
##### Paweł Kapela, Patryk Sajkowski, Maciej Karabin
##### grupa 3. (środa 16:15-19:00)

### Wykorzystane pakiety
Potrzebny będzie program `geckodriver`, będący wersją Firefoxa, którą da się sterować np. biblioteką Selenium.

In [1]:
# Python
import locale
import calendar
from time import sleep
from datetime import datetime

# Sterownik przeglądarki
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

# Selektory
from selenium.webdriver.common.by import By

# Elementy
from selenium.webdriver.support.select import Select

locale.setlocale(locale.LC_TIME, "en_US.UTF-8")

'en_US.UTF-8'

### Stałe konfigurujące proces zgrywania danych
Może zajść potrzeba dostosowania wartości `START_MONTH` i `START_DAY`, jeśli ich język na stronie będzie inny niż polski. Zmienna `END_MONTH` przydaje się przy ponawianiu zgrywania danych w przypadku wystąpienia błędu (nie pobiera się ponownie tych samych danych).

In [2]:
# LOCATION = "brazil/curitiba"
#LOCATION = "colombia/medellin"
# START_MONTH = "September 2009"
# START_DAY = "8 September 2009"

LOCATION = "brazil/natal"
START_MONTH = "March 2010"
START_DAY = "9 March 2010"
END_MONTH = "April 2025"

SOURCE_ADDRESS = f"https://www.timeanddate.com/weather/{LOCATION}/historic"


COOKIES_MORE_OPTIONS_BUTTON_ID = "more-options-btn"
COOKIES_REJECT_ALL_BUTTON_ID = "reject-all-btn"
COOKIES_SAVE_AND_EXIT_BUTTON_SELECTOR = ".qc-cmp2-buttons-desktop > button:nth-child(1)" # istnieje duplikat ID, dlatego selektor

MONTH_SELECT_ID = "month"
DAY_SELECT_ID = "wt-his-select"

### Konfiguracja przeglądarki

In [3]:
options = Options()
options.headless = True
options.set_preference("intl.accept_languages", "en")

driver = webdriver.Firefox(options=options)

### Otworzenie strony, nawigacja po niej i zczytywanie danych
##### Otwarcie i oczekiwanie na wczytanie

In [4]:
driver.get(SOURCE_ADDRESS)
driver.implicitly_wait(5)

##### Odrzucenie ciasteczek

In [5]:
sleep(0.5)
driver.find_element(By.ID, COOKIES_MORE_OPTIONS_BUTTON_ID).click()

sleep(1)
driver.find_element(By.ID, COOKIES_REJECT_ALL_BUTTON_ID).click()

sleep(0.1)
driver.find_element(By.CSS_SELECTOR, COOKIES_SAVE_AND_EXIT_BUTTON_SELECTOR).click()

##### Wczytanie dostępnych opcji w polu wyboru miesiąca i przefiltrowanie ich przez `START_MONTH`

In [6]:
month_select = Select(driver.find_element(By.NAME, MONTH_SELECT_ID))
month_options = [option.text for option in month_select.options]

if "Past 2 Weeks" in month_options:
    month_options.remove("Past 2 Weeks")

start_date = datetime.strptime(START_MONTH, "%B %Y")
month_options = [option for option in month_options if datetime.strptime(option, "%B %Y") >= start_date]

##### Iteracja przez miesiące, ich dni, znajdowanie min. i max. temperatur oraz zapis do pliku CSV na bieżąco

In [7]:
with open(f"{LOCATION.replace("/", "_")}.csv", "w") as out:
    out.write("date;min_temp;max_temp\n")

    for month in month_options:
      month_select = Select(driver.find_element(By.NAME, MONTH_SELECT_ID))
      month_select.select_by_visible_text(month)

      sleep(0.1)

      day_select = Select(driver.find_element(By.ID, DAY_SELECT_ID))
      day_options = [option.text for option in day_select.options]

      if month == START_MONTH or month == END_MONTH:
          year, month_num = datetime.strptime(month, "%B %Y").year, datetime.strptime(month, "%B %Y").month
          last_day = calendar.monthrange(year, month_num)[1]

          start_date_str = START_DAY if month == START_MONTH else "1 " + month
          end_date_str = str(last_day) + " " + month

          start_date = datetime.strptime(start_date_str, "%d %B %Y")
          end_date = datetime.strptime(end_date_str, "%d %B %Y")

          day_options = [option for option in day_options if start_date <= datetime.strptime(option, "%d %B %Y") <= end_date]

      for day in day_options:
          day_select.select_by_visible_text(day)

          sleep(0.5)

          temperatures = []
          temperature_cells = driver.find_elements(By.CSS_SELECTOR, "#wt-his > tbody:nth-child(2) > tr > td:nth-child(3)")

          for cell in temperature_cells:
            n = cell.text

            if n != "N/A":
              temperatures.append(int(n.split(" ")[0]))

          if len(temperatures) == 0:
            sleep(0.05)
            continue

          date = datetime.strptime(day, "%d %B %Y").strftime("%d.%m.%Y")
          min_temp = min(temperatures)
          max_temp = max(temperatures)

          out.write(f"{date};{min_temp};{max_temp}\n")

          sleep(0.05)

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:747:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:559:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElements@chrome://remote/content/marionette/driver.sys.mjs:1822:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


### Zamknięcie przeglądarki

In [15]:
driver.quit()